In [2]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

from torch.autograd import Variable
from torchvision import datasets, models, transforms, utils

import torch.nn.functional as F
import os, glob

import torch.optim as optim
from torch.optim import lr_scheduler

In [3]:
base_dir = '/Users/jooyoungson/P2_fashion_classifier/PyTorch_Ver/dataset_pytorch'

In [4]:
class_num = glob.glob(base_dir + '/train/**')
len(class_num)

20

In [5]:
class_num = glob.glob(base_dir + '/val/**')
len(class_num)

20

In [6]:
trans = transforms.Compose([transforms.Resize((224, 224)), 
                            transforms.ToTensor(), 
                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) #img Tensor로 변경후 normalize

train_dataset = torchvision.datasets.ImageFolder(root=base_dir +'/train', 
                                              transform = trans )
val_dataset = torchvision.datasets.ImageFolder(root=base_dir + '/val', transform = trans)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()        
        #Layer1
        # CONV
            # input_channel_개수: 3
            # output_channel_개수(필터의 개수): 64
            # kernel_size(필터의 사이즈): 3 X 3
            # padding = 1
        # ReLU
        # MaxPool
            # kernel_size = 2
            # stride = 2
        #    Input           -> (1, 3, 224, 224)
        #    layer1.Conv     -> (1, 64, 224, 224)
        #    layer1.Pool     -> (1, 64, 112, 112)
        self.layer1 = nn.Sequential(
                        nn.Conv2d(3, 64, 3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        #Layer2
        # CONV
            # input_channel_개수: 64
            # output_channel_개수(필터의 개수): 32
            # kernel_size(필터의 사이즈): 3 X 3
            # padding = 1
        # ReLU
        # MaxPool
            # kernel_size = 2
            # stride = 2
        #    Input           -> (2, 64, 112, 112)
        #    layer2.Conv     -> (2, 32, 112, 112)
        #    layer2.Pool     -> (2, 32, 56, 56)
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64, 32,  3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        
        #Layer3
        # CONV
            # input_channel_개수: 32
            # output_channel_개수(필터의 개수): 10
            # kernel_size(필터의 사이즈): 4 X 4
            # padding = 1
        # ReLU
        # MaxPool
            # kernel_size = 2
            # stride = 2
        #    Input           -> (2, 32, 56, 56)
        #    layer2.Conv     -> (2, 10, 55, 55)
        #    layer2.Pool     -> (2, 10, 27, 27)
        self.layer3 = nn.Sequential(
                        nn.Conv2d(32, 10, 4, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.layer4 = nn.Sequential(
                        nn.Conv2d(10, 5, 2),
                        nn.Conv2d(5, 1, 2),
                        nn.MaxPool2d(kernel_size=2, padding=1))
        
        # L4 FC 14 * 14 inputs -> 196 outputs
        self.fc1 = torch.nn.Linear(196, 39, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(-1, 196)   # Flatten them for FC
        out = self.fc1(out)
        
        return out

In [9]:
model = CNN()

learning_rate = 0.001
training_epochs = 2
batch_size = 1

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# train my model
total_batch = len(train_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:

        X = X
        Y = Y

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.


/anaconda3/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


[Epoch:    1] cost = 3.06505609
[Epoch:    2] cost = 2.38756919
Learning Finished!


In [11]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [12]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/Users/jooyoungson/Datasets/Fashion'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs
                labels = labels

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        str_epoch = str(epoch)
        torch.save(model.state_dict(), data_dir+"model-"+str_epoch+".pth") # Save checkpoint

        input_tensor = torch.rand(1, 3, 224, 224) #an example input
        script_module = torch.jit.trace(model, input_tensor)
        script_module.save(data_dir+"class1.pt")

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    

    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [14]:
model_ft = models.mobilenet_v2(pretrained=True)
# backbone = model_ft.features
# new_model = nn.Sequential(backbone, nn.AdaptiveAvgPool2d(1000), nn.Linear(1000,39))

model_ft.classifier[1].out_features = 20

In [15]:
#TransferLearning을 위한 가중치 고정
for param in model_ft.features.parameters():
    param.requires_grad = False

In [16]:
learning_rate = 0.001
training_epochs = 2
batch_size = 1

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ft.parameters(), lr=learning_rate)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [17]:
model_ft = train_model(model_ft, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=2)

Epoch 0/1
----------


/anaconda3/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


train Loss: 4.8707 Acc: 0.0607
val Loss: 3.4300 Acc: 0.2467
Epoch 1/1
----------


/anaconda3/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


train Loss: 3.3024 Acc: 0.1877
val Loss: 2.2022 Acc: 0.4167
Training complete in 6m 50s
Best val Acc: 0.416667
